In [ ]:
from attention import AttentionLayer
import pandas as pd
import numpy as np
import scipy as sp
import re
from bs4 import BeautifulSoup
import tensorflow as tf
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense, Bidirectional, Input, Embedding, Dropout
from tensorflow.keras.models import Model
import os
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import nltk.stem
from nltk.corpus import stopwords

In [ ]:
lstmdim = 500
tf.keras.backend.clear_session()
#Encoding Segment
textinput = Input(shape=(MAX_TEXT_LEN,))
textembed = Embedding(len(texttoken.word_index)+1, lstmdim, trainable=True, mask_zero=True)(textinput)
encout1, _, _, _, _ = Bidirectional(LSTM(lstmdim, return_sequences=True, return_state=True))(textembed)
encout1 = Dropout(0.1)(encout1)
_, enc_h, enc_c = LSTM(lstmdim, return_sequences=True, return_state=True)(encout1)
#Decoding Segment
summinput = Input(shape=(None,))
decembed_layer = Embedding(len(summtoken.word_index)+1, lstmdim, trainable=True)
summembed = decembed_layer(summinput)
declstm_layer = LSTM(lstmdim, return_sequences=True, return_state=True)
decout, _, _ = declstm_layer(summembed, initial_state=[enc_h, enc_c])
decdense_layer = Dense(len(summtoken.word_index)+1, activation="softmax")
preds = decdense_layer(decout)
mdl = Model(inputs=[textinput, summinput], outputs=preds)
mdl.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
mdl.summary()

In [ ]:
check = tf.keras.callbacks.ModelCheckpoint("newsbbcseq2seq.h5", save_best_only=True, monitor="val_loss", verbose=True)
hist = mdl.fit([trainX, trainY[:,:-1]], trainY.reshape(trainY.shape[0], trainY.shape[1], 1)[:,1:], epochs=10, callbacks=[check],
               batch_size=16, verbose=True,
               validation_data=([testX, testY[:,:-1]], testY.reshape(testY.shape[0], testY.shape[1], 1)[:,1:]))

In [ ]:
#Run-Time Model Graphs
encode_model = Model(inputs=textinput, outputs=[enc_h, enc_c])
dec_h = Input(shape=(lstmdim,))
dec_c = Input(shape=(lstmdim,))
decinput = Input(shape=(None,))
decembed = decembed_layer(decinput)
output, new_h, new_c = declstm_layer(decembed, initial_state=[dec_h, dec_c])
output = decdense_layer(output)
decode_model = Model(inputs=[decinput, dec_h, dec_c], outputs=[output, new_h, new_c])